In [2]:
import pandas as pd
from itertools import cycle
import os

In [3]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho) 
tbl_ = '\\modelo\\Modelo_score_train.xlsx'

In [4]:
#Tabela de treino:
tbl_train = pd.read_excel(path_arq+tbl_)
analistas_df = tbl_train

In [5]:
#tabela novas demandas:

tbl_test = pd.read_excel(path_arq+'\\cubo_anl\\tbl_full_analitica.xlsx')
novos_chamados_df = tbl_test[['CHAMADO','DT_ABERTURA_CHAMADO','TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE']].drop_duplicates(keep='first')

In [6]:
# Agrupar analistas por módulo e ordenar por rank
analistas_por_modulo = analistas_df.sort_values(by=['MODULO_CHAMADO', 'rank']).groupby('MODULO_CHAMADO')

# Criar iteradores rotativos para cada módulo
iteradores = {modulo: cycle(grupo['IS'].tolist()) for modulo, grupo in analistas_por_modulo}

# Função para distribuir chamados
def distribuir_chamados(novos_chamados_df, iteradores):
    alocacao = []
    for _, chamado in novos_chamados_df.iterrows():
        modulo = chamado['MODULO_CHAMADO']
        if modulo in iteradores:
            analista = next(iteradores[modulo])
            alocacao.append(analista)
        else:
            alocacao.append(None)  # Caso não haja analistas para o módulo
    return alocacao

# Distribuir os novos chamados
novos_chamados_df['IS'] = distribuir_chamados(novos_chamados_df[['TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE']], iteradores)

# Exibir a distribuição
novos_chamados_df.sort_values(['TIPO_CHAMADO','MODULO_CHAMADO']).ffill()
novos_chamados_df['ANOMES_ABERTURA'] = novos_chamados_df['DT_ABERTURA_CHAMADO'].str[0:4] + novos_chamados_df['DT_ABERTURA_CHAMADO'].str[5:7]


# Criando Pivot Table para os gráficos:

pivot_REDIST = novos_chamados_df.pivot_table(
    index='ANOMES_ABERTURA',
    columns='COMPLEXIDADE',
    values='CHAMADO',
    aggfunc='count'
).reset_index().fillna(0)

pivot_REDIST.columns.name = None

pivot_REDIST = pivot_REDIST[pivot_REDIST['ANOMES_ABERTURA'] >= '202401']
pivot_REDIST

In [15]:
# Faz o Export para acompanhar indicador:

pivot_REDIST.to_excel(path_arq+'\\modelo\\alocacao_recursos.xlsx', index=False)

pivot_REDIST.reset_index().drop(columns='index').to_json(path_arq+'\\modelo\\cubo3_alocacao_recursos.json', orient='records', indent=4)

In [ ]:
# Separa a quantidade e tipo de chamados por consultor e envia para eles um alerta por email:

Chamados_por_consultor = novos_chamados_df.groupby(['IS']).agg({
    'CHAMADO': 'count',
    'TIPO_CHAMADO': lambda x: '|'.join(set(x))
}).reset_index()


import win32com.client as win32
import os
# Criar instância do Outlook
outlook = win32.Dispatch('outlook.application')

# Iterar sobre cada funcionário no DataFrame agrupado
for index, row in Chamados_por_consultor.iterrows():
    funcionario = row['IS']
    qtd_chamados = row['CHAMADO']
    tipos_chamados = row['TIPO_CHAMADO']

    try:
        # Criar um novo e-mail
        mail = outlook.CreateItem(0)
        
        # Definir o e-mail do funcionário (ajuste conforme necessário)
        mail.To = f'{funcionario}@softtek.com.br'

        # Assunto do e-mail
        mail.Subject = f'Atenção! Você está com {qtd_chamados} chamados para tratamento!'

        # Corpo do e-mail
        mail.Body = f"""
        Olá {funcionario},

        Você tem {qtd_chamados} chamado(s) pendente(s). Os tipos de chamados são os seguintes:

        {tipos_chamados.replace('|', '\n- ')}

        Por favor, verifique e tome as medidas necessárias.

        Atenciosamente,
        Gestão

        """

        # Enviar o e-mail
        mail.Send()

    except Exception as e:
        print(f'Falha ao enviar e-mail para {funcionario}@softtek.com.br Erro: {str(e)}')

print('E-mails enviados com sucesso!')